In [82]:
import pandas as pd
import pdb
import numpy as np
import itertools
from snakemake.io import expand
import yaml
import os
import sys
import pyfaidx

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from utils import *
from sm_utils import *
from humanized_utils import *

In [38]:
config_file = '../config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [100]:
mouse_fa_file = '../'+expand(config['ref']['fa'])[0]
mouse_gtf_file = '../'+expand(config['ref']['gtf'])[0]
human_fa_file = '../'+expand(config['human_ref']['fa'])[0]

replacement_fa_file = 'hTREM2-R47HKI/seq.fa'

In [11]:
config_tsv = '../231020_config.tsv'
meta_tsv = '../mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe,
                       include_humanized=True)

In [21]:
# gs = [f for f in df.genotype.unique().tolist()]
# for g in gs:
#     print(g)

## hTREM2 pseudochromosome

In [16]:
with open('hTREM2-R47HKI/seq.fa', 'r') as f:
    for line in f:
        if line.startswith('>'):
            continue
        else:
            seq = line.strip()

In [32]:
# mouse stuff
chrom = 'chr17'
start = 48346556
stop = 48352024

if start > stop:
    temp = start
    start = stop
    stop = temp

In [31]:
fa = pyfaidx.Fasta(mouse_fa_file)
seq = fa[chrom][start:stop]
seq.seq

In [63]:
# human stuff -- first verify the strandedness and the coordinates
chrom = 'chr6'
start =  41158764
stop = 41163082
inc_stop = 0
inc_start = -1
complement = True
rev = True

if start > stop:
    temp = start
    start = stop
    stop = temp

In [64]:
fa = pyfaidx.Fasta(human_fa_file)
seq = fa[chrom][start+inc_start:stop+inc_stop]
if complement: 
    seq = seq.complement
else: 
    seq = seq.seq
if rev:
    seq = seq[::-1]
print(len(seq))

4319


In [65]:
print(seq[:22])
print(seq[-22:])

ATGGAGCCTCTCCGGCTGCTCA
TGCAGGGCTGAGAGACACGTGA


In [ ]:
# so this sequence was on the minus strand

In [87]:
# mouse stuff
chrom = 'chr17'
start = 48346556
stop = 48352024
inc_stop = 1
inc_start = 0
complement = False
rev = False

print(abs(start-stop)+1)

if start > stop:
    temp = start
    start = stop
    stop = temp

5469


In [88]:
fa = pyfaidx.Fasta(mouse_fa_file)
seq = fa[chrom][start+inc_start:stop+inc_stop]
if complement: 
    seq = seq.complement
else: 
    seq = seq.seq
if rev:
    seq = seq[::-1]
print(len(seq))

5469


In [96]:
# sequence in mouse gene to get replaced
replaced_seq = seq

In [139]:
# sequence that will replace 
replacement_seq = pyfaidx.Fasta(replacement_fa_file)['hTREM2-R47HKI'][:].seq
# type(replacement_seq)

In [86]:
# get mouse Trem2 sequence to jam the new sequence in
gene = 'Trem2'
gene_seq = get_gene_seq(mouse_fa_file,
                   mouse_gtf_file, 
                   gene,
                   ofile = 'gene_fastas/trem2.fa',
                   slack=3000)

chr17
48343400
48357147


In [89]:
len(gene_seq)

13747

In [114]:
print(replaced_seq[:10])
print(replaced_seq[-10:])

TGGGACCTCT
GGTACGTGAG


In [97]:
anchor_5 = replaced_seq[:20]
anchor_3 = replaced_seq[-20:]

In [98]:
start_inds = [(m.start(), m.end()) for m in re.finditer(anchor_5, gene_seq, re.IGNORECASE)]
assert len(start_inds) == 1

In [99]:
end_inds = [(m.start(), m.end()) for m in re.finditer(anchor_3, gene_seq, re.IGNORECASE)]
assert len(end_inds) == 1

In [94]:
assert gene_seq[start_inds[0][0]:start_inds[0][1]].lower() == anchor_5.lower()
assert gene_seq[end_inds[0][0]:end_inds[0][1]].lower() == anchor_3.lower()

In [125]:
# assert len(gene_seq[start_inds[0][0]:end_inds[0][1]]) == len(replaced_seq)
# temp = gene_seq[start_inds[0][0]:end_inds[0][1]]
# assert temp[:10] == replaced_seq[:10]
# assert temp[-10:] == replaced_seq[-10:]

In [126]:
# gene_seq[:start_inds[0][0]+20][-35:]

In [127]:
# replaced_seq[:10]

In [140]:
# pieces to stitch together
seq_1 = gene_seq[:start_inds[0][0]]
print(seq_1[-1])
print(replaced_seq[0])

seq_2 = replacement_seq
seq_3 = gene_seq[end_inds[0][1]:]
new_seq = seq_1+seq_2+seq_3

A
T


In [141]:
print(len(gene_seq))
print(len(replaced_seq))
print(len(replacement_seq))

new_len = len(gene_seq)-len(replaced_seq)+len(replacement_seq)
print(new_len)
assert new_len == len(new_seq)

13747
5469
4319
12597


In [142]:
ofile = 'hTREM2-R47HKI/pseudochrom.fa'
chr_name = 'hTREM2-R47HKI'
write_chr(new_seq, ofile, chr_name)